## Урок 13. Модель BERT и GPT/ ДЗ


**Задание**

Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

In [1]:
import numpy as np
import pandas as pd

import torch
import torch_directml
from torch import argmax
from torch.nn.functional import softmax
from transformers import BertTokenizerFast, AutoModelForSequenceClassification

In [2]:
from datasets import load_dataset
from transformers import BertTokenizer

In [3]:
corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

In [4]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [5]:
result_df = []

def paraphrase_check(row_numb):
    
    text1 = corpus['train']['text_1'][row_numb]
    text2 = corpus['train']['text_2'][row_numb]
    
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).numpy()
    
    p = proba[0]
    
    result_df.append([text1, text2, p[1]])
    
    return pd.DataFrame(result_df, columns=['текст 1', 'текст 2', 'вероятность парафраза'])

In [6]:
paraphrase_check(0)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301


In [7]:
paraphrase_check(10)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294


In [8]:
paraphrase_check(20)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294
2,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144


In [9]:
paraphrase_check(30)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294
2,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144
3,"Прокуратура пока не решила, обжаловать ли приг...",Защита Васильевой обжалует приговор суда,0.239415


In [10]:
paraphrase_check(40)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294
2,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144
3,"Прокуратура пока не решила, обжаловать ли приг...",Защита Васильевой обжалует приговор суда,0.239415
4,Владимир Путин освободил от должности почти 20...,Путин снял с должностей более 20 руководителей...,0.972911


**Вывод:** предобученная модель работает удовлетворительно

## 
**Задание**

С помощью бертов решить задачу классификации

In [11]:
DEVICE = 0
MODEL = 'blanchefort/rubert-base-cased-sentiment'

device = 'cpu'
if type(DEVICE) is int:
    device = torch_directml.device(DEVICE)
else:
    device = DEVICE

In [12]:
class SentimentClassifier:
    CALSSES = ['Нейтральный', 'Позитивный', 'Негативный']
    
    def __init__(self, model, device='cpu'):
        self._tokenizer = BertTokenizerFast.from_pretrained(model)
        self._model = AutoModelForSequenceClassification.from_pretrained(model)
        self.to(device)
    
    @torch.no_grad()
    def predict(self, text, aggregate=True):
        inputs = self._tokenizer(text,
                                 max_length=512,
                                 padding=True,
                                 truncation=True,
                                 return_tensors='pt')

        inputs = inputs.to(self._device)
        outputs = self._model(**inputs)
        predicted = softmax(outputs.logits, dim=1)
        predicted = argmax(predicted, dim=1).to('cpu').numpy()
        
        if aggregate:
            try:
                return self.CALSSES[predicted[0]]
            except:
                return 'Ошибка определения класса!'
        
        return predicted
    
    def to(self, device):
        self._model = self._model.to(device)
        self._device = device

classifier = SentimentClassifier(MODEL, device=device)

In [13]:
print(classifier.predict('Я обожаю тяжелую музыку!'))

Позитивный


In [14]:
print(classifier.predict('Прекрасный пример полного отсутствия мозга.'))

Негативный


In [15]:
print(classifier.predict('Скоро мы будем делать чат бота.'))

Нейтральный
